In [1]:
# Import normal libraries
from datetime import datetime as dt
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Import own libraries
from data_api.init_db import initialize_db
from data_api.init_db import return_engine
from data_api.db import tickers_to_ticker_ids, get_all_price_data_sets, load_formatted_train_data
from models.model_func import create_train_test_arrays

# Define the data paths
current_dir = os.getcwd()
database_dir = os.path.join(current_dir, "data")
db_filename = "database.db"
model_dir = os.path.join(current_dir, "data/models")
scaler_dir = os.path.join(current_dir, "data/scalers")

# Create the required folders, if they are not available
if not os.path.exists(database_dir):
    os.makedirs(database_dir)
if not os.path.exists(model_dir):
    os.makedirs(model_dir)
if not os.path.exists(scaler_dir):
    os.makedirs(scaler_dir)
if os.path.exists(os.path.join(database_dir, db_filename)):
    # If the database already exits, get a connection to the database
    engine = return_engine(database_dir, db_filename=db_filename)
else:
    # Otherwise, create a new database and initialize it with data
    engine = initialize_db(database_dir,  db_filename=db_filename)

2022-04-22 18:55:12.118998: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-22 18:55:12.119036: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


OpenBLAS blas_thread_init: pthread_create failed for thread 1 of 8: Resource temporarily unavailable
OpenBLAS blas_thread_init: RLIMIT_NPROC 62987 current, 62987 max
OpenBLAS blas_thread_init: pthread_create failed for thread 2 of 8: Resource temporarily unavailable
OpenBLAS blas_thread_init: RLIMIT_NPROC 62987 current, 62987 max
OpenBLAS blas_thread_init: pthread_create failed for thread 3 of 8: Resource temporarily unavailable
OpenBLAS blas_thread_init: RLIMIT_NPROC 62987 current, 62987 max
OpenBLAS blas_thread_init: pthread_create failed for thread 4 of 8: Resource temporarily unavailable
OpenBLAS blas_thread_init: RLIMIT_NPROC 62987 current, 62987 max
OpenBLAS blas_thread_init: pthread_create failed for thread 5 of 8: Resource temporarily unavailable
OpenBLAS blas_thread_init: RLIMIT_NPROC 62987 current, 62987 max
OpenBLAS blas_thread_init: pthread_create failed for thread 6 of 8: Resource temporarily unavailable
OpenBLAS blas_thread_init: RLIMIT_NPROC 62987 current, 62987 max
Open

KeyboardInterrupt: 

# Problem Introduction

From the Yahoo Finance API, we get the 6 different values for each day. High price, low price, open price, close price, adjusted close price and volume. The 5 price data variables are all closely linked together, and only the adjusted close price reflects additional events like divident payouts or stock splits. Therefore, we only consider the adjusted close price (now referred to as "price") and the volume for our models. 

Our goal is to predict future prices based on the past prices and the past volume. We can use the price/volume data of multiple stocks to predict the price/volume data of those multiple stocks to make use of the information hidden in the dependence of the development in those different assets. The problem is, that we have indeed a long timerange of past data, but we do not know anything about future prices (yet). Therefore, we need to split the past data into small sections and consider the earlier part of that section as the past data and the later part as the future. Therefore we can create many training sets of past/future data combinations to use for our training.

# Metrics

We need to define a useful metric or multiple metrics to evaluate the performance of our model and to compare different models with different hyperparameters. The goal of our model is to predict continious values, therefore we have a regression problem. For regression problems, we can use root mean squared error (RMSE) as a metric to optimize our model for. This metric weights larger deviations from the expected results more than lower deviations, but still has a value on the same scale as the output data.

- https://keras.io/api/metrics/
- https://machinelearningmastery.com/custom-metrics-deep-learning-keras-python/

# EDA (Exploratory data analysis)

When we check the data, we encounter one problem: We do not have price and volume data available for arbitrary dates for each asset. For weekends and holidays, there is usually no trading occuring on stock exchanges and therefore no price is determined. These dates without price data vary between different assets, depending on which exchanges they are listed and in which jurisdiction the exchanges are located. For some assets, like Bitcoin, there is nevertheless price data available for all days because there is less regulation about the exchanges and the cryptocurrencies can be easily traded completly digital. Additionally, there is no price data available before the asset was available. In the case of a publicly traded company, this is usually the IPO.
To train a model with different assets, we are therefore likely facing missing values for some assets on some dates. For the first, problem, namely missing days for holidays/weekends, we have two options:
- Drop the data for a specific date, when no price data is missing for at least one asset
- Fill the missing prices with the average price of the days before and after

In the first case, we would potentially reduce our dataset significant, in case we have many different assets from many differen exchanges/jurisdictions. Therefore we take the second option and fill up the missing prices as mentioned.

To handle the second problem, namely missing asset prices because the asset was not publicly traded, it is more difficult to handle this. We can either:
- Limit the oldest date to the date when, the last asset was available
- Assume an average price of the asset for the time before it was available
- Assume a price of 0 before the asset was available
- Assume the price of the first day when the asset was available



In [2]:
start_date=dt(2020,1,1)
end_date=dt.today()
tickers = ["GOOG"] #, "AAPL", "BTC-USD"]
tickers_ids = tickers_to_ticker_ids(engine, tickers)
# Load the data
data = load_formatted_train_data(engine, tickers_ids, start_date, end_date, return_timestamp=False)

NameError: name 'engine' is not defined

# Only for  Google Colab

In [3]:
# data.to_csv("data_big.csv", index = False)

In [4]:
# from google.colab import drive
# drive.mount('/content/drive')

In [5]:
# data_small_path = "/content/drive/MyDrive/Colab Notebooks/data_small.csv"
# data_tiny_path = "/content/drive/MyDrive/Colab Notebooks/data_very_small.csv"
# data_big_path = "/content/drive/MyDrive/Colab Notebooks/data_big.csv"
# data_tiny_path =  "data_very_small.csv"
# data_small_path = "data_small.csv"

In [6]:
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# data = pd.read_csv(data_small_path)
# data.head()
# data.shape

In [7]:
def create_train_test_arrays(n_past, df):
    """
    Takes the data and creates trainX and trainY datasets. n_past values are used to predict the value at index n_past + 1
    """
    # Prepare training and testing data
    # Empty lists to be populated using formatted training data
    trainX = []
    trainY = []
    # Reformat input data into a shape: (n_samples x timesteps x n_features)
    for i in range(n_past, len(df)):
        trainX.append(df[i - n_past:i, 0:df.shape[1]])
        trainY.append(df[i:i + 1])
    trainX, trainY = np.array(trainX), np.array(trainY)
    print("Train dataset was successfully created:")
    print(f"trainX shape == {trainX.shape}")
    print(f"trainY shape == {trainY.shape}")
    return trainX, trainY

# Modelling


To model our problem based on time series data, we need to recognize that the sequence of the data is also containing a lot of information and need to be used in the model. For this type of problem, a long short-term memory (LSTM) model is usually the best fit. With this model, the sequence data is processed in sequence and the data contained in the earlier sequence steps is kept during the whole process and has an influence on the prediction.

If we look at our training, data, we recognize different magnitudes especially when we compare the prices in dollars and the volume data in pieces. Additionally, the prices of stocks are also arbitrary, depending on how many single stock represent the ownership of the full company. Therefore we normalize the dataset to remove these huge ranges of values.

In [8]:
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras import metrics
import tensorflow as tf

In [9]:
# Normalize the dataset
scaler = StandardScaler()
scaler = scaler.fit(data)
df_for_training_scaled = scaler.transform(data)

NameError: name 'data' is not defined

Additional layers -> decrease performance

In [10]:
def get_model(input_shape, output_shape, activation="relu", init_mode='uniform', learning_rate=0.001,beta_1=0.9,beta_2=0.999,amsgrad=False, dropout_rate=0.2):
    """
    Returns a predefined model object
    """
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, beta_1=beta_1, beta_2=beta_2, amsgrad=amsgrad)
    model = Sequential()
    model.add(LSTM(64, activation=activation, input_shape=(
        input_shape[1], input_shape[2]), return_sequences=True, kernel_initializer=init_mode))
    model.add(LSTM(32, activation=activation, return_sequences=False, kernel_initializer=init_mode))
    model.add(Dropout(dropout_rate))
    # If non-negative return values are required, this should be accomplished by the layers in the network. Anyway, with only non-negative input values, negative output values are very unlikely.
    # The relu activation function only returns positive values. -> This returns the same values for each predicted date
    model.add(Dense(output_shape[2]))  # ,  W_constraint=nonneg()))
    model.compile(optimizer=optimizer, loss='mean_squared_error',metrics=[
        metrics.RootMeanSquaredError(),
    ])
    return model

An important decision to make is how many past data sets (each data set representing one date) are used to predict the following data set. This influences how we create our training data set. We tested out different values and the result is, that a quite good solution is 120.

In [11]:
# n_past_set = [30, 60, 90, 120, 180, 200]
# results = {}

# for n_past in n_past_set:
#     print(f"Calculating for {n_past} ....")
#     # n_past = 60
#     trainX, trainY = create_train_test_arrays(n_past=n_past, df=df_for_training_scaled)

#     def get_model(input_shape, output_shape):
#         """
#         Returns a predefined model object
#         """
#         model = Sequential()
#         model.add(LSTM(64, activation='relu', input_shape=(
#             input_shape[1], input_shape[2]), return_sequences=True))
#         model.add(LSTM(32, activation='relu', return_sequences=False))
#         model.add(Dropout(0.2))
#         # If non-negative return values are required, this should be accomplished by the layers in the network. Anyway, with only non-negative input values, negative output values are very unlikely.
#         # The relu activation function only returns positive values. -> This returns the same values for each predicted date
#         model.add(Dense(output_shape[2]))  # ,  W_constraint=nonneg()))
#         model.compile(optimizer='adam', loss='mse',metrics=[
#             metrics.RootMeanSquaredError(),
#         ])
#         return model

#     model = get_model(input_shape=trainX.shape, output_shape=trainY.shape)
#     # model.summary()

#     # fit the model
#     history = model.fit(trainX, trainY, epochs=30, batch_size=64, validation_split=0.1, verbose=0)
    
#     results[n_past] = history

In [12]:
# # history.history.keys()
# f = plt.figure()
# f.set_figwidth(15)
# f.set_figheight(10)

# for n_past in n_past_set:
#     plt.plot(results[n_past].history['root_mean_squared_error'], label = str(n_past))
#     plt.legend(loc="upper right")
#     plt.title('model root_mean_squared_error')
#     plt.ylabel('value')
#     plt.xlabel('epoch')

# plt.show()

In [13]:
n_past = 120
trainX, trainY = create_train_test_arrays(n_past=n_past, df=df_for_training_scaled)

NameError: name 'df_for_training_scaled' is not defined

# Hyperparameter tuning

Now that we have our dataset ready, we need to find out what is a good model to predict the data set for the next price based on the last 120 data sets. After trying out several amounts of layers, we conclude that having two layers with 64 and 32 nodes leads to a quite good result. We choose the adam optimizer because it is known to be a good and efficient optimizer for regression problems.

We can use GridSearchCV to further optimize our model. First, we check different combinations of batch_sizes and epochs to see which combinations leads to the best result.

Furthermore, we can check different parameters for our adam activation function

In [14]:
def evaluate_gridsearch(result):
    """
    Prints an formatted output for the GridSeachCV result
    """
    print('Best Estimator: %s' % result.best_estimator_)
    print('Best Hyperparameters: %s' % result.best_params_)
    print(f'Best Score: {result.best_score_}') # score according to the metric we passed in refit

In [15]:
# https://github.com/adriangb/scikeras
from scikeras.wrappers import KerasClassifier, KerasRegressor
# Wrap the model to use the GridSearchCV library from sklearn
model = KerasRegressor(model=get_model, input_shape=trainX.shape, output_shape=trainY.shape, activation="relu", init_mode='uniform', optimizer = "adam", dropout_rate=0.2)

NameError: name 'trainX' is not defined

## Batch size and Epochs
We test different values for batch size and epoch using GridSearch to find the best combination

In [16]:
search_space = {
    "batch_size" : [4, 8, 16, 32, 64], # smaller batch size require a longer training process, larger batch sizes require more powerfull hardware
    "epochs" : [10, 25, 50, 100, 200, 250]
}
from sklearn.model_selection import GridSearchCV
GS = GridSearchCV(estimator = model,
                param_grid = search_space,
                scoring = "neg_root_mean_squared_error", # With Sklearn, there is a convention that a higher score is always the better score, therefore root mean squared error is available as a negated score
                cv = 5,
                refit = True,
                n_jobs = 1, # Use all available threads
            #    error_score="raise",
                verbose = 2)
history = GS.fit(trainX, trainY[:,0,:])

NameError: name 'model' is not defined

In [17]:
evaluate_gridsearch(history)

NameError: name 'history' is not defined

After running the GridSearchCV algorithm, we come to the best parameter combination:
- batch_size: 32
- epochs: 250

After running different combinations, we also realized that increasing the amount of epochs may improve the model a bit more, but it increases computation time extensively.
The best score for the neg_root_mean_squared_error is -0.6563984733875825, which leads to a root_mean_squared_error of 0.6563984733875825.

## Hyperparameters for adam optimizer
The adam optimizer has several different parameters which we can test and combine to find a more optimal solution for our problem than with the default adam parameters.

In [18]:
adam_parameters = {
    "optimizer__learning_rate" : [0.001,  0.0001, 0.01, 0.1],
    "optimizer__beta_1" : [0.8, 0.9, 1.0],
    "optimizer__beta_2" : [0.89, 0.99, 1.0],
    "optimizer__amsgrad" : [True, False],
    # already optimized parameters, which we like to use again for out training
    "batch_size" : [32],
    "epochs" : [250]
}
from sklearn.model_selection import GridSearchCV
GS_adam = GridSearchCV(estimator = model,
                param_grid = adam_parameters,
                scoring = "neg_root_mean_squared_error", # With Sklearn, there is a convention that a higher score is always the better score, therefore root mean squared error is available as a negated score
                cv = 5,
                refit = True,
                n_jobs = 1, # Use all available threads
                error_score="raise",
                verbose = 1)
history_adam = GS_adam.fit(trainX, trainY[:,0,:])

NameError: name 'model' is not defined

In [19]:
evaluate_gridsearch(history_adam)

NameError: name 'history_adam' is not defined

We come to the conclusion, that our best parameters are:
- optimizer__amsgrad: False
- optimizer__beta_1: 0.8
- optimizer__beta_2: 0.89
- optimizer__learning_rate: 0.01

The best score for the neg_root_mean_squared_error is -0.8073114933087524, which leads to a root_mean_squared_error of 0.8073114933087524.

# Results

After testing different LSTM networks, hyperparameters and training parameters either manually, or by the use of GridSearch, we can come to a conclusion for a quite good model:

- 120 days as input data to predict for day 121
- 2 LSTM layers with 64 and 32 nodes respectively
- Batch size for training of 32 and 250 training epochs. More epochs improve our result, but they lead to a lot of computation time. Use less epochs may be also sufficient.

# Improvements

There are possibilties to spend time and computation capacity to further improve the model. Furthermore, there are many more different optimizers as well as activation functions available, which have each individually countless different parameter combinations. These could be evaluated to see if there is a possibility to still improve the model.
Also, other techniques except GridSearch could be used to test different sets of parameters. There is a library called RandomSearch which is similar to GridSearch, but creates more random combination by using ranges of values for the parameters.
A different option woult be the usage of Genetic algorithms. These algorithms apply a model similar to evolution to the models to find a optimal set of paramter

# Conclusion/Reflection

Training machine learning models is a very time intensive process, if it is done wrong. Therefore I come to the following conclusions to speed up development in the future:
- Test all scripts etc. with small amounts of data to verify that everything runs correct to avoid breakages after hours of model training and grid search
- Run scripts overnight to not waste time waiting for the result
- Run scripts on a server so that it can run without occuping local resources and wearing down a laptops CPU